In [5]:
import numpy as np
import pandas as pd
import requests
import datetime
import tqdm
import time
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt

geckterminal_url = "https://api.geckoterminal.com/api/v2"
ton_url = "https://tonapi.io/v2"
stonfi_url = "https://api.ston.fi/v1"
dedust_url = "https://api.dedust.io/v2"
ton_api_key = "AHNEEFDML7QU2FAAAAAF7S4ESSY3QQLM6EDTLGATFSSAG5PQHGQUIFRPZKE7ST6MIVN3B2A"

def get_pickle(path):
    out = None
    with open(path, 'rb') as f:
        out = pickle.load(f)
    return out

## Достаем все пулы на DeDust

### Исследование пулов

In [46]:
url = dedust_url + '/pools'
print(f'get {url}')
pools = requests.get(url)
pools.json()[0]

get https://api.dedust.io/v2/pools


{'address': 'EQDRV4AlGgMx3T1mG3H1tode8_6FXajBMFjd8UHDZ9dqzxbC',
 'lt': '43644368000005',
 'totalSupply': '1000',
 'type': 'volatile',
 'tradeFee': '0.4',
 'assets': [{'type': 'jetton',
   'address': 'EQBiHSLVzoicc-LQPDXJc_rF-WRft0aFihccFduA6fQ1ynYM',
   'metadata': {'name': 'Baron',
    'symbol': 'BARO',
    'image': 'https://assets.dedust.io/images/baro.webp',
    'decimals': 9}},
  {'type': 'jetton',
   'address': 'EQBl3gg6AAdjgjO2ZoNU5Q5EzUIl8XMNZrix8Z5dJmkHUfxI',
   'metadata': {'name': 'Lavandos',
    'symbol': 'LAVE',
    'image': 'https://assets.dedust.io/images/lave.webp',
    'decimals': 9}}],
 'lastPrice': None,
 'reserves': ['825', '122'],
 'stats': {'fees': ['0', '0'], 'volume': ['0', '0']}}

In [47]:
none_supply_pools = 0
none_last_price_pools = 0
types_of_pools = defaultdict(int)
lts = []
supply = []
for pool in pools.json():
    types_of_pools[pool['type']]+=1
    lts.append(int(pool['lt']))
    supply.append(int(pool['totalSupply']))
    if pool['totalSupply'] == '0':
        none_supply_pools += 1
    if pool['lastPrice'] is None:
        none_last_price_pools += 1
print(f'Pools: {len(pools.json())}, with types {list(types_of_pools.items())}')
print(f'Pools with TotalSupply=0: {none_supply_pools}')
print(f'Pools with LastPrice=0: {none_last_price_pools}')

Pools: 20932, with types [('volatile', 20924), ('stable', 8)]
Pools with TotalSupply=0: 2110
Pools with LastPrice=0: 20585


In [48]:
pd.DataFrame(supply).value_counts()

0                                  
1000                                   14915
0                                       2110
1000000000001000                         125
100000000001000                          113
1000001000                                92
                                       ...  
7547440177148                              1
7452560929977                              1
7423976658023                              1
7371783702742                              1
43990694442740576290346711940594142        1
Name: count, Length: 2124, dtype: int64

In [49]:
pd.DataFrame(lts).value_counts()

0             
0                 2096
46610847000001      30
46610856000001      19
46610822000001      19
46610833000001      11
                  ... 
45948795000005       1
45948727000013       1
45948696000009       1
45948676000005       1
47157166000001       1
Name: count, Length: 18534, dtype: int64

In [50]:
for pool in pools.json():
    if pool['type'] == 'stable':
        print(pool['assets'][0]['metadata']['name'], '--',pool['assets'][1]['metadata']['name'] if pool['assets'][1]['metadata'] is not None else pool['assets'][1])

TON Bridge USDT -- TON Bridge USDC
TON Bridge USDT -- Tether USD
Toncoin -- {'type': 'jetton', 'address': 'EQAcHW1VR8AmIlKyC5P9SBVKiVjEwWY1IXpI4w-vXlZKSU7I', 'metadata': None}
Toncoin -- {'type': 'jetton', 'address': 'EQBNo5qAG8I8J6IxGaz15SfQVB-kX98YhKV_mT36Xo5vYxUa', 'metadata': None}
Toncoin -- Ton Whales liquid staking token
Toncoin -- Hipo Staked TON
Toncoin -- Staked TON by bemo
ARBUZ -- ARBUZ


### Достаем все адреса токенов из всех пулов на DeDust

In [51]:
dedust_jettons = []
none_metadata = 0
if pools:
    for i, pool in enumerate(pools.json()):
        if pool['assets'][0]['type'] == 'jetton':
            dedust_jettons.append((pool['assets'][0]['metadata'], pool['assets'][0]['address']))
        if pool['assets'][1]['type'] == 'jetton':
            dedust_jettons.append((pool['assets'][1]['metadata'], pool['assets'][1]['address']))
else:
    print(pools)

In [52]:
dedust_jettons_addr = [j for i,j in dedust_jettons]
dedust_jettons_metadata = [i for i,j in dedust_jettons]
dedust_jettons_addr_dist = set(dedust_jettons_addr)
print("jettons in pools:", len(dedust_jettons))
print("different jettons in pools:", len(set(dedust_jettons_addr)))
print("None metadata jettons in pools:", dedust_jettons_metadata.count(None))

jettons in pools: 22834
different jettons in pools: 19324
None metadata jettons in pools: 20235


In [53]:
# dedust_jettons_to_metadata = {}
dedust_jettons_to_metadata = get_pickle('dicts/dedust_jettons_to_metadata.pkl')

In [54]:
def get_response(jetton):
    response = requests.get(ton_url + f'/jettons/{jetton}')
    if response:
        dedust_jettons_to_metadata[jetton] = response.json()
    else:
        time.sleep(1)
        get_response(jetton)

In [33]:
for jetton in tqdm.tqdm(list(dedust_jettons_addr_dist), total=len(dedust_jettons_addr_dist)):
    if dedust_jettons_to_metadata.get(jetton) is None or dedust_jettons_to_metadata.get(jetton) == {'error': 'rate limit: free tier'}:
        get_response(jetton)

 78%|███████▊  | 14094/18135 [00:01<00:00, 11756.30it/s]


KeyboardInterrupt: 

In [29]:
with open('dicts/dedust_jettons_to_metadata.pkl', 'wb') as f:
    pickle.dump(dedust_jettons_to_metadata, f)

In [12]:
dedust_jettons_to_metadata['EQAam5Dl28BMaBtXKdDYkT_DekS6a_zKvOVoH0g4SDQcPFdv']

{'mintable': True,
 'total_supply': '500000000000000',
 'admin': {'address': '0:48ddc9e0faa49aa7767069720e67b793e70de744c11bd56b822f5cf891fc48e8',
  'is_scam': False,
  'is_wallet': True},
 'metadata': {'address': '0:1a9b90e5dbc04c681b5729d0d8913fc37a44ba6bfccabce5681f483848341c3c',
  'name': 'Bonk',
  'symbol': 'BONK',
  'decimals': '9',
  'image': 'https://cache.tonapi.io/imgproxy/cOMlJuViiVXDCkAghnyNj7plX8pAZ9pv3WhklvebpTY/rs:fill:200:200:1/g:no/aHR0cHM6Ly9yYXcuZ2l0aHVidXNlcmNvbnRlbnQuY29tL3RvbmtlZXBlci9vcGVudG9uYXBpL21hc3Rlci9wa2cvcmVmZXJlbmNlcy9tZWRpYS90b2tlbl9wbGFjZWhvbGRlci5wbmc.webp'},
 'verification': 'none',
 'holders_count': 6}

In [13]:
list(dedust_jettons_to_metadata.values()).count({'error': 'rate limit: free tier'}), list(dedust_jettons_to_metadata.values()).count(None) 

(464, 0)

### Первые транзакции по адресам токенов

In [55]:
requests.get(ton_url + '/blockchain/accounts/EQD2R36ustW2sAJBlWsfEWn36mds32QPImD5tsZNX4j5N6dw/transactions', params={'sort_order': 'asc', 'limit': '1'}).json()

{'transactions': [{'hash': 'fe6dc0f0f3fdf35984194de9ec24ee92f8d3c630b4a9da4d18b40223ad897137',
   'lt': 44483237000003,
   'account': {'address': '0:f6477eaeb2d5b6b00241956b1f1169f7ea676cdf640f2260f9b6c64d5f88f937',
    'is_scam': False,
    'is_wallet': False},
   'success': True,
   'utime': 1707406074,
   'orig_status': 'nonexist',
   'end_status': 'active',
   'total_fees': 9972616,
   'end_balance': 33394000,
   'transaction_type': 'TransOrd',
   'state_update_old': '90aec8965afabb16ebc3cb9b408ebae71b618d78788bc80d09843593cac98da4',
   'state_update_new': 'e9cb13646398c49be33fdd0e0414cee3ed9e623f124789627d5edcb72a7dacd1',
   'in_msg': {'msg_type': 'int_msg',
    'created_lt': 44483237000002,
    'ihr_disabled': True,
    'bounce': True,
    'bounced': False,
    'value': 250000000,
    'fwd_fee': 14489444,
    'ihr_fee': 0,
    'destination': {'address': '0:f6477eaeb2d5b6b00241956b1f1169f7ea676cdf640f2260f9b6c64d5f88f937',
     'is_scam': False,
     'is_wallet': False},
    'sour

In [56]:
# dedust_jettons_to_create_time = {}
# dedust_jettons_to_create_time_errors = {}
dedust_jettons_to_create_time = get_pickle("dicts/dedust_jettons_to_create_time.pkl")
dedust_jettons_to_create_time_errors = get_pickle("dicts/dedust_jettons_to_create_time_errors.pkl")


In [15]:

try:
    for i, token_addr in tqdm.tqdm(enumerate(set(dedust_jettons_addr_dist) - set(dedust_jettons_to_create_time.keys())), total=len(set(dedust_jettons_addr_dist) - set(dedust_jettons_to_create_time.keys()))):
        if dedust_jettons_to_create_time.get(token_addr) is None:
            token_transactions = requests.get(
                ton_url + '/blockchain/accounts/' + token_addr + '/transactions', 
                params={"sort_order": "asc", "limit": 100},
                headers={"Authorization": f"Bearer {ton_api_key}"})
            time.sleep(4)
            if not token_transactions:
                print(f"bad_request: {token_transactions.text}")
                dedust_jettons_to_create_time_errors[token_addr] = f"bad_request: {token_transactions.text}"
                continue
            token_transactions = token_transactions.json()
            if len(token_transactions['transactions']) == 0:
                dedust_jettons_to_create_time_errors[token_addr] = f"empty transactions"
                continue
            j = 0
            while token_transactions['transactions'][j]['success'] != True:
                if j == 99 or j == len(token_transactions['transactions']) - 1:
                    dedust_jettons_to_create_time_errors[token_addr] = f"no success transactions out of {j+1} first"
                    continue
                j+=1 
            # print(datetime.datetime.fromtimestamp(token_transactions['transactions'][j]['utime']), j)
            # print(a['transactions'][0])
            dedust_jettons_to_create_time[token_addr] = token_transactions['transactions'][j]
except Exception as e:
    print(e)
    with open('dicts/dedust_jettons_to_create_time.pkl', 'wb') as f:
        pickle.dump(dedust_jettons_to_create_time, f)
    with open('dicts/dedust_jettons_to_create_time_errors.pkl', 'wb') as f:
        pickle.dump(dedust_jettons_to_create_time_errors, f)

  0%|          | 0/5096 [00:00<?, ?it/s]

 15%|█▍        | 763/5096 [54:49<5:09:13,  4.28s/it]

bad_request: {"error":"inMessage not found for tx 87ff4bc068f7ff69967b44d3082b1473e12abaf020f65de960c880e3e2b1fce0"}


 18%|█▊        | 937/5096 [1:07:25<5:17:03,  4.57s/it]

bad_request: {"error":"inMessage not found for tx a9796b8a041cf9c92c1a51e028c766c542fe8db0a8f30aa85f5998d1034d11b2"}


 22%|██▏       | 1120/5096 [1:20:35<4:41:21,  4.25s/it]

bad_request: {"error":"inMessage not found for tx 163715bfe48611c11c8ab7d4f49cd355fc73e46e35fcc3ce0f3e2e9496742a0c"}


 40%|███▉      | 2019/5096 [7:55:30<12:04:40, 14.13s/it]


KeyboardInterrupt: 

In [16]:
with open('dicts/dedust_jettons_to_create_time.pkl', 'wb') as f:
    pickle.dump(dedust_jettons_to_create_time, f)
with open('dicts/dedust_jettons_to_create_time_errors.pkl', 'wb') as f:
    pickle.dump(dedust_jettons_to_create_time_errors, f)

In [57]:
len(dedust_jettons_addr_dist), len(dedust_jettons_to_create_time.keys()), len(dedust_jettons_to_create_time_errors.keys())

(19324, 17811, 10015)

In [58]:
dedust_jettons_to_create_time_errors

{'EQDUUeVL3hE4PVEBQKgt5uhvVbdFl6-_y2wMH949RiQcRY0z': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQD6ZT5B2V0n4QzWH1yBacInRD3cCBU0yc7SUn9xNZR7gdaB': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQDJsLQ0X2V4EUcUSPJlcLsfnvGpCtAhwoTSKWs8aYn989UJ': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQAOhLqgW79Kn94PEFr6FxtO1x1KnJrqblHaPr4XrNfiWINm': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBqIGgBFOa2CAMS0nMtxf6DOZSjKwFZcRg9wTTcme0sRc4Y': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBqGJRj90aixYPsS-iQGzF614-tfkDZaY3iVCxOc0ETq9kY': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQC0c1UQCIpL7gGfVIiYscIgzOxl02Rr9sVVIo5YRFSBZNJp': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBTc1isZGs7mZva7Rcv-dmPQC1QS2iB2Acg2K9V6cm0ttST': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQBqNo8-lfKRUINFXFkiqPs3hmBdA5SOkx3yJ1qXt3NH-XYb': 'bad_request: {"error":"rate limit: limit for tier"}',
 'EQDUByoXDbCgcKhrQ9UvzpObl9

## Достаем все адреса токенов из всех пулов на Ston.fi

In [3]:
url = stonfi_url + '/pools'
print(f'get {url}')
pools = requests.get(url)
stonfi_tokens = set()
if pools:
    pools = pools.json()['pool_list']
    for i, jetton in enumerate(pools):
        stonfi_tokens.add(jetton['token0_address'])
        stonfi_tokens.add(jetton['token1_address'])
else:
    print(pools)

print(len(stonfi_tokens))

get https://api.ston.fi/v1/pools
17043


In [84]:
pools_tokens = dedust_tokens.union(stonfi_tokens)
print(len(pools_tokens))

23945


## токены с Ston.fi

In [56]:
error_tokens = []
success_tokens = []
for i, token_addr in tqdm.tqdm(enumerate(stonfi_tokens), total=len(stonfi_tokens)):
    token_transactions = requests.get(ton_url + '/blockchain/accounts/' + token_addr + '/transactions', params={"sort_order": "asc", "limit": 100})
    if not token_transactions:
        error_tokens.append({"addr": token_addr, "reason": f"bad_request: {token_transactions}"})
        continue
    token_transactions = token_transactions.json()
    if len(token_transactions['transactions']) == 0:
        error_tokens.append({"addr": token_addr, "reason": f"empty transactions"})
        continue
    j = 0
    while token_transactions['transactions'][j]['success'] != True:
        if j == 99 or j == len(token_transactions['transactions']) - 1:
            error_tokens.append({"addr": token_addr, "reason": f"no success transactions out of {j} first"})
            continue
        j+=1
    # print(datetime.datetime.fromtimestamp(token_transactions['transactions'][j]['utime']), j)
    # print(a['transactions'][0])
    success_tokens.append(token_transactions['transactions'][j])
with open('success_tokens.pkl', 'wb') as f:
    pickle.dump(success_tokens, f)
with open('error_tokens.pkl', 'wb') as f:
    pickle.dump(error_tokens, f)


100%|██████████| 10756/10756 [1:11:26<00:00,  2.51it/s] 


In [ ]:
pickle.load()

In [66]:
len(dedust_jettons_to_create_time.keys()), len(set(dedust_jettons_to_create_time.keys())), list(dedust_jettons_to_create_time.values())[0]

(17811,
 17811,
 {'hash': '2b4db117f2f73f7d87202f615583684de96b9f8c43e9783cb40ac00117bdaed4',
  'lt': 46336960000001,
  'account': {'address': '0:3965c81d7792a7ba031f862b10a6e1376d99a4f50a57e66f603393c7503a2e05',
   'is_scam': False,
   'is_wallet': False},
  'success': True,
  'utime': 1715009125,
  'orig_status': 'nonexist',
  'end_status': 'active',
  'total_fees': 3991179,
  'end_balance': 43351200,
  'transaction_type': 'TransOrd',
  'state_update_old': '90aec8965afabb16ebc3cb9b408ebae71b618d78788bc80d09843593cac98da4',
  'state_update_new': 'c2ed4046c9940592996d9bd3a15cb99a221f4cbc0816035510d86b8260c6067c',
  'in_msg': {'msg_type': 'int_msg',
   'created_lt': 46336956000002,
   'ihr_disabled': True,
   'bounce': True,
   'bounced': False,
   'value': 250000000,
   'fwd_fee': 5592043,
   'ihr_fee': 0,
   'destination': {'address': '0:3965c81d7792a7ba031f862b10a6e1376d99a4f50a57e66f603393c7503a2e05',
    'is_scam': False,
    'is_wallet': False},
   'source': {'address': '0:8846630

In [67]:
data=pd.DataFrame(list(map(lambda x: (x['account']['address'], datetime.datetime.fromtimestamp(x['utime']).year, datetime.datetime.fromtimestamp(x['utime']).month), 
                      dedust_jettons_to_create_time.values())), columns=['address', 'year', 'month'])
data.groupby(['year', 'month']).count()


address
year month         
2022 6            2
     7            2
     8            1
     9            2
     10           8
     11          16
     12          14
2023 1           34
     2           26
     3           10
     4           24
     5           25
     6           21
     7           26
     8           31
     9           52
     10          39
     11          63
     12         148
2024 1         1345
     2          905
     3         1599
     4         5562
     5         6107
     6         1749

## токены с DeDust с названием

In [121]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-05-01T12:34:56", 'until': '2024-05-20T23:59:59'})
all_tokens_may = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_may)

7333

In [122]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-04-01T00:00:01", 'until': '2024-05-01T00:00:01'})
all_tokens_april = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_april)

5710

In [123]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-03-01T00:00:01", 'until': '2024-04-01T00:00:01'})
all_tokens_march = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_march)

3306

In [124]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-02-01T00:00:01", 'until': '2024-03-01T00:00:01'})
all_tokens_feb = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_feb)

1484

In [125]:
a = requests.get(stonfi_url + "/stats/pool", params={'since': "2024-01-01T00:00:01", 'until': '2024-02-01T00:00:01'})
all_tokens_jan = set(map(lambda x: x['base_symbol'],a.json()['stats'])).union(set(map(lambda x: x['quote_symbol'],a.json()['stats'])))
len(all_tokens_jan)

1326

январь - 1326

февраль - 1484 (+158)

март - 3306 (+1822)

апрель - 5710 (+2404)

май - 7333 (+1623)

In [134]:
all_tokens_jan

{'Stay Toned',
 'JCOIN',
 'Tborkkk',
 'TRSHC',
 'CRAB',
 'Duck Dark',
 'PONTON',
 'cMaG',
 'FLOTON',
 'BONTON',
 'Like2',
 'EST',
 'MYCRO',
 'KRYPTON',
 'pcltst',
 'THF',
 'testfest',
 'tBONE',
 'DI',
 'TSN',
 'THub',
 'Vrl',
 'TONY',
 'SquidGrow',
 'donttest',
 'END',
 'TT2',
 'TONOG',
 'STAN',
 'Clown',
 'Tonfi',
 'ALy',
 'OnlyFans',
 'LEGOAI',
 'Starship',
 'JHLC',
 'test2',
 'BOZO',
 'Tshb',
 'gigachadd',
 'DrEgg',
 'OVER',
 'TOOTHLESS',
 'CODE',
 'Pork',
 'NewTON',
 'NTR',
 'TALI',
 'PUTON',
 'TYoshi',
 'SYNTH',
 'TRB',
 'TMyro',
 'WHALE',
 'DOGLABS',
 'MOT',
 'ToNoT',
 'PWNAO',
 'DRE',
 'DOGS',
 'GROYP',
 'pTON-tsTON LP',
 'CUNO',
 'SANIC',
 'ULT',
 'jMAWA',
 'ShS',
 'test3ttt',
 'BOBO',
 'DOGEMEMES',
 'CCS',
 'FlintsTONes',
 'KERMIT',
 'BRB',
 'Pokemon',
 'TGDOG',
 'tPIG',
 'CAKE',
 'DTON',
 '$PORNHUB',
 'f',
 'TONyStark',
 'PUMBA',
 'X',
 'a',
 'DAOXPX',
 'TONEY',
 'TPillz',
 'BANANA',
 'LAVE',
 'bbb',
 'Loony Tons',
 'DOGEMOON',
 'Akimbo',
 'BabyShark',
 'Test',
 'TONKY',
 'FC

## Достаем транзакции по токенам

In [1]:
durev_master_addr = 'EQB02DJ0cdUD4iQDRbBv4aYG3htePHBRK1tGeRtCnatescK0'
my_addr = 'UQD3WUMJjkCFi1Izw6EuitqArI7BX3ecWDmHGyrmKISqZXji'

In [6]:
result = requests.get(ton_url + '/blockchain/accounts/' + durev_master_addr + '/transactions', params={"sort_order": "desc", "limit": 100})

In [7]:
result.json()['transactions'][0]

{'hash': 'e51bf74b1fa41aa8dbed5d1b44cdf78c50cc4e6c395b679b94b018e9eda6d89c',
 'lt': 51344463000001,
 'account': {'address': '0:74d8327471d503e2240345b06fe1a606de1b5e3c70512b5b46791b429dab5eb1',
  'is_scam': False,
  'is_wallet': False},
 'success': True,
 'utime': 1732815872,
 'orig_status': 'active',
 'end_status': 'active',
 'total_fees': 2975169,
 'end_balance': 117671944660,
 'transaction_type': 'TransOrd',
 'state_update_old': '53c7c404992ceadd67dc18edce6760a7ef7e22aea3c9f461f83acf829483230b',
 'state_update_new': '8cdf76ada6f28e9e8878090fa5fdb6018bb45a93090ea6bb19b888a5f2ddb781',
 'in_msg': {'msg_type': 'int_msg',
  'created_lt': 51344460000002,
  'ihr_disabled': True,
  'bounce': True,
  'bounced': False,
  'value': 24414400,
  'fwd_fee': 390403,
  'ihr_fee': 0,
  'destination': {'address': '0:74d8327471d503e2240345b06fe1a606de1b5e3c70512b5b46791b429dab5eb1',
   'is_scam': False,
   'is_wallet': False},
  'source': {'address': '0:04e3241c49038d3e74f1c084f586fbe2c34eb9f56aa99aae4

In [64]:
history = requests.get(ton_url + '/accounts/' + my_addr + '/jettons/'+"0:16a73dbf1b434ac651b656f8056e06463edf18d6a7b47068fee18c3905f99847"+"/history", params={"limit": 100})

In [65]:
history.json()

{'events': [{'event_id': '98eaef25c00e489f8d629693b31cf983a914f55cbd925915f90bca70ba958655',
   'account': {'address': '0:f75943098e40858b5233c3a12e8ada80ac8ec15f779c5839871b2ae62884aa65',
    'is_scam': False,
    'is_wallet': True},
   'timestamp': 1713106940,
   'actions': [{'type': 'JettonTransfer',
     'status': 'ok',
     'JettonTransfer': {'sender': {'address': '0:779dcc815138d9500e449c5291e7f12738c23d575b5310000f6a253bd607384e',
       'name': 'STON.fi Dex',
       'is_scam': False,
       'is_wallet': False},
      'recipient': {'address': '0:f75943098e40858b5233c3a12e8ada80ac8ec15f779c5839871b2ae62884aa65',
       'is_scam': False,
       'is_wallet': True},
      'senders_wallet': '0:43b0d4ab5df3308d49a2c9fdfd80887c7e5c158e434b12620ba06facb7abfab9',
      'recipients_wallet': '0:94cc1ea41f99459cf10a582b6dc588a4c2fb73d0222ed563682b0f3345adc9db',
      'amount': '65701379155863',
      'comment': 'Call: StonfiSwapOk',
      'jetton': {'address': '0:16a73dbf1b434ac651b656f8056